# Model Training and Evaluation
Recommended to run this notebook on Google Colab.

In [ ]:
#@title Kaggle API Configuration
#@markdown Enter your Kaggle API token here
!mkdir /root/.kaggle/

import json
KAGGLE_USERNAME = "" #@param {type:"string"}
KAGGLE_KEY = "" #@param {type:"string"}
data = {
  "username": KAGGLE_USERNAME,
  "key": KAGGLE_KEY,
}
with open('/root/.kaggle/kaggle.json','w') as f:
  f.write(json.dumps(data))

#@markdown Go to https://github.com/Kaggle/kaggle-api#api-credentials for details

# Download the dataset

In [ ]:
# For Training
!kaggle datasets download -d christofel04/fire-detection-dataset
!mkdir /content/Dataset/train
!/bin/7z x /content/fire-detection-dataset.zip -o/content/Dataset/train

#For Validation
!kaggle datasets download -d phylake1337/fire-dataset
!mkdir /content/Dataset/validation
!/bin/7z x /content/fire-dataset.zip -o/content/Dataset/validation

 99% 425M/431M [00:04<00:00, 116MB/s] 
100% 431M/431M [00:04<00:00, 103MB/s]
mkdir: cannot create directory ‘/content/Dataset/train’: No such file or directory

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 451808002 bytes (431 MiB)

Extracting archive: /content/fire-detection-dataset.zip
--
Path = /content/fire-detection-dataset.zip
Type = zip
Physical Size = 451808002

  0%      2% 153 - Fire Dataset PCD/Test/Neutral/464.jpg                                                  5% 224 - Fire Dataset PCD/Test/Neutral/535.jpg                                                

# Model Definition

In [ ]:
import imghdr
import os
import math
import numpy as np
from keras.models import load_model
from keras.optimizers import SGD
import keras.utils as image
from matplotlib import pyplot as plt
from keras import Model
from keras.applications import InceptionV3
from keras.applications.inception_v3 import preprocess_input as inception_preprocess_input
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
class CustomInceptionModel:
    def __init__(self, classes_names, model_path=None):
        """
        Initialises the model with required configurations or loads a pre-trained model

        :param dataset_path: path to the root of the dataset.
        :param classes_names: names of the classes.
        """
        self.classes = classes_names
        self.nbr_classes = len(classes_names)
        if model_path:
            self.model = load_model(model_path)
        else:
            self.model = self.create_inception_based_model(self.nbr_classes)
        self.model.summary()

    def create_inception_based_model(self, nbr_classes):
        """
        Creates an Inception-based model with fixed weights. Custom top layer is added with tuneable parameters to perform classification.

        :param nbr_classes: number of classes.
        :return: the model.
        """
        # weights are pre-trained with imagenet
        base_model = InceptionV3(include_top=False, weights='imagenet', pooling='max', input_shape=(224, 224, 3))

        x = base_model.output
        x = Dense(2048, activation='relu', name='fc_1')(x)
        x = Dense(1024, activation='relu', name='fc_2')(x)
        predictions = Dense(nbr_classes, activation='softmax', name='fc_class')(x)
        model = Model(inputs=base_model.inputs, outputs=predictions)

        # by default only the fc layers are trainable (layers in base_model are not trainable)
        for layer in base_model.layers:
            layer.trainable = False

        return model

    def train_model(self,
                    train_path,
                    valid_path,
                    save_path="best_trained_save.h5",
                    learning_rate=0.001,
                    percentage=0.9,
                    nbr_epochs=10,
                    batch_size=32):
        """
        Creates and train a simpler InceptionV3-based model on the fire images dataset or fine-tunes an pre-trained model
        with a custom learning rate. Some layers of the model can be frozen for training.

        :param learning_rate: when fine-tuning, the learning rate can be specified.
        :param percentage: percentage of samples to be used for training. Must be in [0,1].
        :param nbr_epochs: number of epochs.
        :param batch_size: number of batches.
        """

        #  Freeze every layer but layers fc_1, fc_2 and fc_class at the end of the network.
        for layer in self.model.layers:
            if layer.name != 'fc_1' and layer.name != 'fc_2' and layer.name != 'fc_class':
                layer.trainable = False

        model_save_folder = "model-saves/"

        # create save path
        if not os.path.exists(model_save_folder):
            os.makedirs(model_save_folder)

        model_save_path = model_save_folder + save_path

        # saves the model when validation accuracy improves, overwrites previously saved model
        save_on_improve = ModelCheckpoint(model_save_path,
                                          monitor='val_accuracy',
                                          save_best_only=True,
                                          save_weights_only=False,
                                          mode='max',
                                          verbose=1)

        # callbacks
        cb = [save_on_improve]

        # loss is categorical since we are classifying
        self.model.compile(loss='categorical_crossentropy', optimizer="sgd", metrics=['accuracy'])

        data_generator = ImageDataGenerator(dtype='float32', rescale= 1./255.)

        train_generator = data_generator.flow_from_directory(train_path,
                                                             batch_size = batch_size,
                                                             target_size = (224, 224),
                                                             class_mode = 'categorical')

        valid_generator = data_generator.flow_from_directory(valid_path,
                                                             batch_size = batch_size,
                                                             target_size = (224, 224),
                                                             class_mode = 'categorical')

        nbr_train_samples = train_generator.samples
        nbr_val_samples = valid_generator.samples

        # fitting the model
        history = self.model.fit(
            train_generator,
            steps_per_epoch=math.ceil(nbr_train_samples / batch_size),
            epochs=nbr_epochs,
            validation_data=valid_generator,
            validation_steps=math.ceil(nbr_val_samples / batch_size),
            callbacks=cb, verbose=1)

    def predict(self, image_path):
        """
        Returns the predictions on a specific image

        :param image_path: path of the image to be predicted
        """

        img = image.load_img(image_path, target_size=(224, 224, 3))

        processed_img = image.img_to_array(img)
        processed_img = np.expand_dims(processed_img, axis=0)

        # preprocess the image (optional)
        # processed_img = inception_preprocess_input(processed_img)

        predictions = self.model.predict(processed_img)[0]
        return predictions

    def evaluate_model(self, val_path):
        """
        evaluates the model on images provided in folder a dataset.

        :param val_path: path to the validation dataset.
        """

        data_generator = ImageDataGenerator(dtype='float32', rescale= 1./255.)
        val_generator = data_generator.flow_from_directory(val_path,
                                                        batch_size = 16,
                                                        target_size = (224, 224),
                                                        class_mode = 'categorical')
        nbr_val_samples = val_generator.samples
        metrics = self.model.evaluate(val_generator,
                                      steps=math.ceil(nbr_val_samples / 16),
                                      max_queue_size=10,
                                      workers=1,
                                      use_multiprocessing=True,
                                      verbose=1)

        # return the metrics
        return {name : metric for (name, metric) in zip(self.model.metrics_names, metrics)}

# Create Model

In [ ]:
model = CustomInceptionModel(classes_names = ['Fire','Neutral'])

87910968/87910968 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     

# Train and Evaluate Model

In [ ]:
model.train_model("/content/Dataset/train/Fire Dataset PCD/Train", "/content/Dataset/train/Fire Dataset PCD/Test", nbr_epochs=10)

Found 2320 images belonging to 2 classes.
Found 331 images belonging to 2 classes.
Epoch 1/10
73/73 [==============================] - ETA: 0s - loss: 0.5618 - accuracy: 0.9315
Epoch 1: val_accuracy improved from -inf to 0.95468, saving model to model-saves/best_trained_save.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


73/73 [==============================] - 31s 227ms/step - loss: 0.5618 - accuracy: 0.9315 - val_loss: 0.1305 - val_accuracy: 0.9547
Epoch 2/10
73/73 [==============================] - ETA: 0s - loss: 0.0555 - accuracy: 0.9815
Epoch 2: val_accuracy improved from 0.95468 to 0.95770, saving model to model-saves/best_trained_save.h5
73/73 [==============================] - 13s 181ms/step - loss: 0.0555 - accuracy: 0.9815 - val_loss: 0.1280 - val_accuracy: 0.9577
Epoch 3/10
73/73 [==============================] - ETA: 0s - loss: 0.0336 - accuracy: 0.9905
Epoch 3: val_accuracy did not improve from 0.95770
73/73 [==============================] - 12s 165ms/step - loss: 0.0336 - accuracy: 0.9905 - val_loss: 0.1274 - val_accuracy: 0.9577
Epoch 4/10
73/73 [==============================] - ETA: 0s - loss: 0.0214 - accuracy: 0.9966
Epoch 4: val_accuracy did not improve from 0.95770
73/73 [==============================] - 12s 164ms/step - loss: 0.0214 - accuracy: 0.9966 - val_loss: 0.1311 - val_

In [ ]:
model.evaluate_model("/content/Dataset/validation/fire_dataset")

Found 999 images belonging to 2 classes.
63/63 [==============================] - 37s 592ms/step - loss: 0.1936 - accuracy: 0.9449


{'loss': 0.1936112344264984, 'accuracy': 0.9449449181556702}

# Save and Convert Model
Converting a keras model into a tensorflow.js model to run it on a nodejs environment.

In [ ]:
from google.colab import drive
drive.mount("/content/Drive")

Mounted at /content/Drive


In [ ]:
model.model.save("/content/Drive/MyDrive/College/EcoHackathon/BestModel.keras")

In [ ]:
# !pip3 install tensorflowjs
!tensorflowjs_converter --input_format=keras_saved_model --output_format=tfjs_layers_model /content/Drive/MyDrive/College/EcoHackathon/BestModel.keras /content/Drive/MyDrive/College/EcoHackathon/jsmodel.tfjs

2023-09-20 07:13:30.720106: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-20 07:13:36.656885: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
2023-09-20 07:14:10.258906: W tensorflow/c/c_api.cc:304] Operation '{name:'batch_normalization_20/beta/Assign' id:788 op device:{requested: '', assigned: ''} def:{{{node batch_normalization_20/beta/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](batch_normalization_20/beta, batch_normalization_20/be